# UDP Video Server 실습 노트북

이 노트북은 `udp_server_test.py`를 **이해하기 쉬운 형태**로 재구성한 자료입니다.
카메라 프레임을 JPEG로 압축하고, Base64로 인코딩해 UDP로 전송하는 흐름을 단계별로 다룹니다.

## 1) UDP와 Socket 핵심 개념

- **Socket**: 네트워크 통신의 끝점(endpoint)
- **UDP (User Datagram Protocol)**: 연결을 유지하지 않고 데이터그램 단위로 전송
- **TCP 대비 UDP 특징**
  - 빠르고 가벼움
  - 패킷 손실/순서 뒤바뀜 가능
  - 실시간 영상/센서 스트리밍에 자주 사용

이 코드에서는 서버가 UDP 소켓을 열고, 클라이언트의 요청을 받으면 카메라 프레임을 계속 전송합니다.

In [ ]:
import base64
import socket
import time

import cv2
import imutils

# UDP 수신 버퍼 크기
BUFF_SIZE = 65536

# 네트워크 설정
HOST_IP = "192.168.0.52"  # 실제 LAN IP로 변경
PORT = 9999
SOCKET_ADDRESS = (HOST_IP, PORT)

# 카메라 설정
CAMERA_DEVICE = "/dev/jetcocam0"
FRAME_WIDTH = 400
JPEG_QUALITY = 80

print("SOCKET_ADDRESS =", SOCKET_ADDRESS)

## 2) UDP 서버 소켓 만들기

- `socket.AF_INET`: IPv4 사용
- `socket.SOCK_DGRAM`: UDP 소켓
- `SO_RCVBUF`: 수신 버퍼 크기 설정
- `bind`: 서버 IP/포트에 소켓 연결

In [ ]:
def create_udp_server(host_ip: str, port: int, buff_size: int = BUFF_SIZE):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_RCVBUF, buff_size)
    server_socket.bind((host_ip, port))
    print("[INFO] Listening at:", (host_ip, port))
    return server_socket

server_socket = create_udp_server(HOST_IP, PORT)

## 3) 카메라 열기

`cv2.VideoCapture`로 카메라 장치를 열고, 프레임을 읽을 수 있는지 확인합니다.

In [ ]:
vid = cv2.VideoCapture(CAMERA_DEVICE)

if not vid.isOpened():
    raise RuntimeError(f"카메라를 열 수 없습니다: {CAMERA_DEVICE}")

print("[INFO] Camera opened:", CAMERA_DEVICE)

## 4) 프레임 인코딩 후 UDP 전송

전송 순서:
1. 프레임 읽기
2. 리사이즈
3. JPEG 압축 (`cv2.imencode`)
4. Base64 인코딩
5. `sendto`로 클라이언트에 전송

In [ ]:
def stream_video_over_udp(server_socket, vid, frame_width=400, jpeg_quality=80):
    fps, st, frames_to_count, cnt = 0, 0, 20, 0

    while True:
        # 클라이언트가 먼저 메시지를 보내야 주소(client_addr)를 알 수 있음
        msg, client_addr = server_socket.recvfrom(BUFF_SIZE)
        print("[INFO] Got connection from", client_addr, "msg=", msg[:20])

        while vid.isOpened():
            ok, frame = vid.read()
            if not ok:
                print("[WARN] 프레임을 읽지 못했습니다.")
                break

            frame = imutils.resize(frame, width=frame_width)

            encoded, buffer = cv2.imencode(
                ".jpg", frame, [cv2.IMWRITE_JPEG_QUALITY, jpeg_quality]
            )
            if not encoded:
                print("[WARN] JPEG 인코딩 실패")
                continue

            # bytes -> base64 bytes
            message = base64.b64encode(buffer)
            server_socket.sendto(message, client_addr)

            frame = cv2.putText(
                frame,
                f"FPS: {fps}",
                (10, 40),
                cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,
                0.7,
                (0, 0, 255),
                2,
            )
            cv2.imshow("TRANSMITTING VIDEO", frame)

            key = cv2.waitKey(1) & 0xFF
            if key == ord("q"):
                print("[INFO] q 입력으로 종료합니다.")
                return

            if cnt == frames_to_count:
                try:
                    fps = round(frames_to_count / (time.time() - st))
                    st = time.time()
                    cnt = 0
                except Exception:
                    pass
            cnt += 1


## 5) 실행 셀

아래 셀을 실행하면 스트리밍이 시작됩니다.
종료는 영상 창에서 `q` 키를 누르면 됩니다.

In [ ]:
try:
    stream_video_over_udp(server_socket, vid, FRAME_WIDTH, JPEG_QUALITY)
finally:
    vid.release()
    server_socket.close()
    cv2.destroyAllWindows()
    print("[INFO] 리소스 정리 완료")

## 6) 트러블슈팅

- 검은 화면/카메라 실패: `CAMERA_DEVICE` 경로 확인 (`/dev/jetcocam0`)
- 통신 안 됨: `HOST_IP`가 실제 같은 네트워크 대역의 IP인지 확인
- 속도 저하: `FRAME_WIDTH`를 줄이거나 `JPEG_QUALITY`를 낮추기
- UDP 특성상 패킷 손실은 정상적으로 발생할 수 있음